# Uses interaction to push triples to the brain

Before running, start GraphDB and make sure that there is a sandbox repository

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Importing the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.dummies.text_to_triple as ttt

## Standard initialisation of a scenario

In [5]:
##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images and rdf are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

Directory  ../../data/2021-11-04-10:52:06  Created 
Directory  ../../data/2021-11-04-10:52:06/image  Created 


## Specifying the BRAIN

In [6]:
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)
replier = LenkaReplier()

2021-11-04 10:52:31,386 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-04 10:52:35,521 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents
2021-11-04 10:52:35,521 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


In [ ]:
#### Small sequence to learn name of speaker
initial_prompt = f"{choice(GREETING)} {choice(ASK_NAME)} {HUMAN_NAME}?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get name from person  
utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
textSignal = d_util.create_text_signal(scenario, utterance)
scenario.append_signal(textSignal)

HUMAN_NAME = ttt.get_your_name(utterance)
HUMAN_ID = HUMAN_NAME
chat = Chat(HUMAN_ID)

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = f"{choice(TALK_TO_ME)}"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

#### Get input and loop
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    ###### Getting the next input signals
    utterance = input('\n')
    print(HUMAN_NAME + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    #### Process input and generate reply
    reply = talk.process_text_and_reply(scenario, place_id, location, HUMAN_ID, textSignal, chat, replier, my_brain)
    print(AGENT + ": " + reply)
    textSignal = d_util.create_text_signal(scenario, reply)
    scenario.append_signal(textSignal)


Leolani2: Nice to see you! I would like to know your name! Stranger?



 My name is Piek


Stranger: My name is Piek
2021-11-04 10:52:56,267 -     INFO - cltl.triple_extraction.api.Chat (My_Name_Is_Piek)   000 - << Start of Chat with My_Name_Is_Piek >>
Leolani2: Tell me something, anything!



 I am from Amsterdam


My_Name_Is_Piek: I am from Amsterdam
2021-11-04 10:53:07,483 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-04 10:53:07,484 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-04 10:53:07,489 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-04 10:53:08,952 -     INFO - cltl.triple_extraction.api.Chat (My_Name_Is_Piek)   001 - My_Name_Is_Piek: "I am from Amsterdam"
2021-11-04 10:53:10,477 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: my-name-is-piek_be-from_amsterdam [agent_->_location])
2021-11-04 10:53:10,572 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-11-04 10:53:13,017 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. play sport - 15 gaps as object: e.g. write-by book
Leolani2: Let me ask you something. What types of location orInstance like amsterdam do agent or

...

Leolani2: Sorry, did not get that.



 I like pizza?


My_Name_Is_Piek: I like pizza?
2021-11-04 10:54:58,028 -     INFO - cltl.triple_extraction.api.Chat (My_Name_Is_Piek)   008 - My_Name_Is_Piek: "I like pizza?"
2021-11-04 10:54:58,082 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: my-name-is-piek_like_pizza [agent_->_food])
2021-11-04 10:54:58,166 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: existing subject - new object 
2021-11-04 10:55:00,517 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Gaps: 26 gaps as subject: e.g. favorite interest - 15 gaps as object: e.g. be-parent-of person
Leolani2: I am curious. What types of food orInstance like pizza do agent orInstance usually like



 What do I like?


My_Name_Is_Piek: What do I like?
2021-11-04 10:55:09,074 -     INFO - cltl.triple_extraction.api.Chat (My_Name_Is_Piek)   009 - My_Name_Is_Piek: "What do I like?"
2021-11-04 10:55:09,078 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in question: My_Name_Is_Piek_like_? [agent_->_])
Leolani2: I wouldn't know!


In [ ]:
#scenario.scenario.end = datetime.now().microsecond
scenarioStorage.save_scenario(scenario)

## End of notebook